In [71]:
import requests
import pandas as pd
import tweepy
import json
import time
from tweepy import TweepError

## Data Wrangling Process
<ol>
    <li><strong> Gather; </strong></li>
    <li><strong> Assess; </strong></li>
    <li><strong> Clean. </strong></li>
</ol>

### 1. Gather the Data

#### twitter-archive-enhanced.csv

In [13]:
df_twitter_archive = pd.read_csv("twitter-archive-enhanced.csv", sep=',')

In [16]:
df_twitter_archive.shape

(2356, 17)

In [17]:
df_twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [15]:
df_twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### image-predictions.tsv

In [4]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

In [7]:
response = requests.get(url)
if (response.status_code==200):
    with open('image-predictions.tsv', 'wb') as file:
        file.write(response.content)

In [20]:
img_pred_df = pd.read_csv('image-predictions.tsv', sep='\t')

In [21]:
img_pred_df.shape

(2075, 12)

In [19]:
img_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [11]:
img_pred_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### tweet_json.txt

In [45]:
import twitter_config

In [46]:
twitter_config.access_token_secret

'cDC3yasRWXWtlROHbM4fUzvzWKJlkbhZgVqEM2BFGrAjv'

In [63]:
auth = tweepy.OAuthHandler(twitter_config.consumer_key, twitter_config.consumer_secret)

auth.set_access_token(twitter_config.access_token, twitter_config.access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [72]:
# get all unique tweet_ids from the twitter-archive-enhanced.csv dataframe
tweet_ids = set(df_twitter_archive['tweet_id'].unique())

with open('all_tweet.json', 'wt') as file:
    for tweet_id in tweet_ids:
        
        print ('Reading tweet {}'.format(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, file)
            file.write('\n')
            time.sleep(1)
        except TweepError as e:
            print ('Error reading tweet {}'.format(tweet_id))
            print(e)

Reading tweet 891815181378084864
Reading tweet 892420643555336193
Reading tweet 891327558926688256
Reading tweet 891689557279858688
Reading tweet 890971913173991426
Reading tweet 890729181411237888
Reading tweet 890609185150312448
Reading tweet 890006608113172480
Reading tweet 886054160059072513
Reading tweet 883838122936631299
Reading tweet 841439858740625411
Reading tweet 887473957103951883
Reading tweet 828801551087042563
Reading tweet 861005113778896900
Reading tweet 786233965241827333
Reading tweet 777953400541634568
Reading tweet 676821958043033607
Reading tweet 670792680469889025
Reading tweet 670046952931721218
Reading tweet 705102439679201280
Reading tweet 668643542311546881
Reading tweet 667550882905632768
Reading tweet 674042553264685056
Reading tweet 673708611235921920
Reading tweet 671789708968640512
Reading tweet 801285448605831168
Reading tweet 801115127852503040
Reading tweet 798682547630837760
Reading tweet 749317047558017024
Reading tweet 784431430411685888
Reading tw

Reading tweet 703268521220972544
Reading tweet 700029284593901568
Reading tweet 698989035503689728
Reading tweet 698635131305795584
Reading tweet 676975532580409345
Reading tweet 678798276842360832
Reading tweet 728986383096946689
Reading tweet 728046963732717569
Reading tweet 716439118184652801
Reading tweet 708479650088034305
Reading tweet 677565715327688705
Reading tweet 676946864479084545
Reading tweet 676237365392908289
Reading tweet 675003128568291329
Reading tweet 673656262056419329
Reading tweet 668955713004314625
Reading tweet 697616773278015490
Reading tweet 745057283344719872
Reading tweet 707014260413456384
Reading tweet 689877686181715968
Reading tweet 678675843183484930
Reading tweet 676590572941893632
Reading tweet 832682457690300417
Reading tweet 738156290900254721
Reading tweet 675047298674663426
Reading tweet 667393430834667520
Reading tweet 732585889486888962
Reading tweet 667453023279554560
Reading tweet 667509364010450944
Reading tweet 675483430902214656
Reading tw

Reading tweet 757596066325864448
Reading tweet 752309394570878976
Reading tweet 693642232151285760
Reading tweet 674690135443775488
Reading tweet 786051337297522688
Reading tweet 868622495443632128
Reading tweet 868880397819494401
Reading tweet 667200525029539841
Reading tweet 669661792646373376
Reading tweet 685547936038666240
Reading tweet 791406955684368384
Reading tweet 668297328638447616
Reading tweet 730211855403241472
Reading tweet 872820683541237760
Reading tweet 870804317367881728
Reading tweet 865359393868664832
Reading tweet 861383897657036800
Reading tweet 846139713627017216
Reading tweet 844973813909606400
Reading tweet 708119489313951744
Reading tweet 706291001778950144
Reading tweet 705975130514706432
Reading tweet 678396796259975168
Reading tweet 683742671509258241
Reading tweet 707387676719185920
Reading tweet 704871453724954624
Reading tweet 871515927908634625
Reading tweet 866450705531457537
Reading tweet 863432100342583297
Reading tweet 712085617388212225
Reading tw

Reading tweet 705223444686888960
Reading tweet 703611486317502464
Reading tweet 702684942141153280
Reading tweet 697575480820686848
Reading tweet 697242256848379904
Reading tweet 696488710901260288
Reading tweet 687664829264453632
Reading tweet 678380236862578688
Reading tweet 676603393314578432
Reading tweet 691483041324204033
Reading tweet 679062614270468097
Reading tweet 678643457146150913
Reading tweet 673345638550134785
Reading tweet 672834301050937345
Reading tweet 671891728106971137
Reading tweet 671544874165002241
Reading tweet 671497587707535361
Reading tweet 668587383441514497
Reading tweet 667495797102141441
Reading tweet 666430724426358785
Reading tweet 672239279297454080
Reading tweet 800859414831898624
Reading tweet 758854675097526272
Reading tweet 753398408988139520
Reading tweet 695446424020918272
Reading tweet 798644042770751489
Reading tweet 762464539388485633
Reading tweet 759047813560868866
Reading tweet 686003207160610816
Reading tweet 691820333922455552
Reading tw

Reading tweet 702217446468493312
Reading tweet 854365224396361728
Reading tweet 847962785489326080
Reading tweet 886736880519319552
Reading tweet 884441805382717440
Reading tweet 874296783580663808
Reading tweet 852311364735569921
Reading tweet 872122724285648897
Reading tweet 850753642995093505
Reading tweet 845677943972139009
Reading tweet 873337748698140672
Reading tweet 879008229531029506
Reading tweet 872261713294495745
Error reading tweet 872261713294495745
[{'code': 144, 'message': 'No status found with that ID.'}]
Reading tweet 869596645499047938
Reading tweet 861288531465048066
Reading tweet 855138241867124737
Reading tweet 866334964761202691
Reading tweet 669749430875258880
Reading tweet 667065535570550784
Reading tweet 692187005137076224
Reading tweet 671182547775299584
Reading tweet 780074436359819264
Reading tweet 684097758874210310
Reading tweet 786286427768250368
Reading tweet 855860136149123072
Reading tweet 674265582246694913
Reading tweet 788552643979468800
Reading tw

Reading tweet 878604707211726852
Reading tweet 774314403806253056
Reading tweet 879415818425184262
Reading tweet 765222098633691136
Reading tweet 822462944365645825
Reading tweet 847606175596138505
Reading tweet 820690176645140481
Reading tweet 838831947270979586
Reading tweet 747103485104099331
Reading tweet 770414278348247044
Reading tweet 815745968457060357
Reading tweet 776218204058357768
Reading tweet 779124354206535695
Reading tweet 757729163776290825
Reading tweet 703356393781329922
Reading tweet 701545186879471618
Reading tweet 700002074055016451
Reading tweet 783695101801398276
Reading tweet 772877495989305348
Reading tweet 745712589599014916
Reading tweet 695767669421768709
Reading tweet 687818504314159109
Reading tweet 674014384960745472
Reading tweet 673686845050527744
Reading tweet 670786190031921152
Reading tweet 681523177663676416
Reading tweet 666293911632134144
Reading tweet 752682090207055872
Reading tweet 737826014890496000
Reading tweet 803773340896923648
Reading tw

Reading tweet 741099773336379392
Reading tweet 738891149612572673
Reading tweet 748568946752774144
Reading tweet 780192070812196864
Reading tweet 778624900596654080
Reading tweet 725729321944506368
Reading tweet 841320156043304961
Reading tweet 838476387338051585
Reading tweet 842115215311396866
Reading tweet 837366284874571778
Error reading tweet 837366284874571778
[{'code': 144, 'message': 'No status found with that ID.'}]
Reading tweet 836380477523124226
Reading tweet 826598365270007810
Reading tweet 775898661951791106
Reading tweet 690005060500217858
Reading tweet 679862121895714818
Reading tweet 774757898236878852
Reading tweet 770655142660169732
Reading tweet 760656994973933572
Reading tweet 748977405889503236
Reading tweet 686386521809772549
Reading tweet 680801747103793152
Reading tweet 695816827381944320
Reading tweet 667171260800061440
Reading tweet 674644256330530816
Reading tweet 854482394044301312
Reading tweet 765371061932261376
Reading tweet 679877062409191424
Reading tw

Reading tweet 715220193576927233
Reading tweet 724004602748780546
Reading tweet 712438159032893441
Reading tweet 712092745624633345
Reading tweet 710844581445812225
Reading tweet 708810915978854401
Reading tweet 843235543001513987
Reading tweet 714251586676113411
Reading tweet 707315916783140866
Reading tweet 704847917308362754
Reading tweet 677687604918272002
Reading tweet 670822709593571328
Reading tweet 670804601705242624
Reading tweet 666055525042405380
Reading tweet 681694085539872773
Reading tweet 685268753634967552
Reading tweet 673711475735838725
Reading tweet 672984142909456390
Reading tweet 799774291445383169
Reading tweet 778774459159379968
Reading tweet 697259378236399616
Reading tweet 674805413498527744
Reading tweet 667924896115245057
Reading tweet 877556246731214848
Reading tweet 849668094696017920
Reading tweet 709409458133323776
Reading tweet 708853462201716736
Reading tweet 707776935007539200
Reading tweet 706681918348251136
Reading tweet 703425003149250560
Reading tw

Reading tweet 699788877217865730
Reading tweet 748346686624440324
Reading tweet 675149409102012420
Reading tweet 668992363537309700
Reading tweet 669567591774625800
Reading tweet 866816280283807744
Error reading tweet 866816280283807744
[{'code': 144, 'message': 'No status found with that ID.'}]
Reading tweet 773670353721753600
Reading tweet 699060279947165696
Reading tweet 669993076832759809
Reading tweet 673689733134946305
Reading tweet 676617503762681856
Reading tweet 668932921458302977
Reading tweet 699079609774645248
Reading tweet 681339448655802368
Reading tweet 790698755171364864
Reading tweet 749996283729883136
Reading tweet 857214891891077121
Reading tweet 678969228704284672
Reading tweet 675849018447167488
Reading tweet 675710890956750848
Reading tweet 668113020489474048
Reading tweet 692905862751522816
Reading tweet 817502432452313088
Reading tweet 812747805718642688
Error reading tweet 812747805718642688
[{'code': 144, 'message': 'No status found with that ID.'}]
Reading tw

Reading tweet 763183847194451968
Reading tweet 685198997565345792
Reading tweet 668537837512433665
Reading tweet 747242308580548608
Reading tweet 781308096455073793
Reading tweet 778650543019483137
Reading tweet 760290219849637889
Reading tweet 753294487569522689
Reading tweet 708738143638450176
Reading tweet 853639147608842240
Reading tweet 848324959059550208
Reading tweet 877201837425926144
Reading tweet 876484053909872640
Reading tweet 874057562936811520
Reading tweet 852553447878664193
Reading tweet 888202515573088257
Error reading tweet 888202515573088257
[{'code': 144, 'message': 'No status found with that ID.'}]
Reading tweet 872620804844003328
Reading tweet 870374049280663552
Reading tweet 862096992088072192
Reading tweet 881906580714921986
Reading tweet 879674319642796034
Reading tweet 874434818259525634
Reading tweet 870656317836468226
Reading tweet 856602993587888130
Error reading tweet 856602993587888130
[{'code': 144, 'message': 'No status found with that ID.'}]
Reading tw

In [ ]:
tweepy.Cursor(api.user_timeline, id="666020888022790149")

In [49]:
# Only iterate through the first 200 statuses
for status in tweepy.Cursor(api.user_timeline, tweet_id="666020888022790149").items(10):
    print(status)

Status(_api=<tweepy.api.API object at 0x000001FB13355C40>, _json={'created_at': 'Wed Feb 19 14:39:39 +0000 2020', 'id': 1230139892681846784, 'id_str': '1230139892681846784', 'text': '🛫🏔️🇮🇸 Tá procurando um destino exótico, incrível e pouco conhecido dos brasileiros?\n. \nPromoção de passagem para… https://t.co/NrSrGj2jEU', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/NrSrGj2jEU', 'expanded_url': 'https://twitter.com/i/web/status/1230139892681846784', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [114, 137]}]}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1221060097381752832, 'id_str': '1221060097381752832', 'name': 'Passagens para o Mundo', 'screen_name': 'PassagensMundo', 'locatio

In [50]:
id_list = ['666020888022790149']
tweets = api.statuses_lookup(id_list) # id_list is the list of tweet ids
for i in tweets:
    print(i.text)
    


Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj


In [ ]:
with open('tweet_json.txt', 'w') as outfile:
    for id in tw.tweet_id:
        try:
            tweet = api.get_status(id, tweet_mode='extended')
            json.dump(tweet._json, outfile)
            outfile.write('\n') 
        except:
            # this was printed a lot of times
            print("ERROR cannot get tweet")